In [ ]:
import os
from dotenv import load_dotenv, find_dotenv



In [2]:
_ = load_dotenv(find_dotenv())

In [ ]:
## 获取folder_path下所有文件路径，储存在file_paths里
file_paths = []
folder_path='files'
for root, dirs, files in os.walk(folder_path):
    for file in files:
        file_path=os.path.join(root, file)
        file_paths.append(file_path)
print(file_paths[0:3])

['files\\pumpkin_book.pdf', 'files\\必修一-Prompt Engineering For Developers\\1. 简介.md', 'files\\必修一-Prompt Engineering For Developers\\2. 提示原则 Guidelines.ipynb']


In [8]:
from langchain.document_loaders.pdf import PyMuPDFLoader
from langchain.document_loaders.markdown import UnstructuredMarkdownLoader

In [9]:
# 遍历文件路径并把实例化的loader存放在loaders里
loaders = []

for file_path in file_paths:

    file_type = file_path.split('.')[-1]
    if file_type == 'pdf':
        loaders.append(PyMuPDFLoader(file_path))
    elif file_type == 'md':
        loaders.append(UnstructuredMarkdownLoader(file_path))

In [10]:
# 下载文件并存储到text
texts = []

for loader in loaders: texts.extend(loader.load())

In [11]:
text = texts[1]
print(f"每一个元素的类型：{type(text)}.", 
    f"该文档的描述性数据：{text.metadata}", 
    f"查看该文档的内容:\n{text.page_content[0:]}", 
    sep="\n------\n")

每一个元素的类型：<class 'langchain_core.documents.base.Document'>.
------
该文档的描述性数据：{'producer': 'xdvipdfmx (20200315)', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-11-17T15:20:45+00:00', 'source': 'files\\pumpkin_book.pdf', 'file_path': 'files\\pumpkin_book.pdf', 'total_pages': 196, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': "D:20231117152045-00'00'", 'page': 1}
------
查看该文档的内容:
前言
“周志华老师的《机器学习》（西瓜书）是机器学习领域的经典入门教材之一，周老师为了使尽可能多的读
者通过西瓜书对机器学习有所了解, 所以在书中对部分公式的推导细节没有详述，但是这对那些想深究公式推
导细节的读者来说可能“不太友好”，本书旨在对西瓜书里比较难理解的公式加以解析，以及对部分公式补充
具体的推导细节。”
读到这里，大家可能会疑问为啥前面这段话加了引号，因为这只是我们最初的遐想，后来我们了解到，周
老师之所以省去这些推导细节的真实原因是，他本尊认为“理工科数学基础扎实点的大二下学生应该对西瓜书
中的推导细节无困难吧，要点在书里都有了，略去的细节应能脑补或做练习”。所以...... 本南瓜书只能算是我
等数学渣渣在自学的时候记下来的笔记，希望能够帮助大家都成为一名合格的“理工科数学基础扎实点的大二
下学生”。
使用说明
• 南瓜书的所有内容都是以西瓜书的内容为前置知识进行表述的，所以南瓜书的最佳使用方法是以西瓜书
为主线，遇到自己推导不出来或者看不懂的公式时再来查阅南瓜书；
• 对于初学机器学习的小白，西瓜书第1 章和第2 章的公式强烈不建议深究，简单过一下即可，等你学得
有点飘的时候再回来啃都来得及；
•

In [14]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50)

split_docs = text_splitter.split_documents(texts)

In [17]:
from langchain_openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings()

In [ ]:
import os

# 指定文件夹路径
persist_directory = 'vector_db/chroma'

# 检查目录是否存在，如果不存在则创建
if not os.path.exists(persist_directory):
    os.makedirs(persist_directory)

# 现在你可以在这个路径下使用 Chroma 存储数据
print(f"Directory '{persist_directory}' is ready!")

#如果该文件夹中有文件，需要手动删除


Directory 'vector_db/chroma' is ready!


In [23]:
from langchain.vectorstores.chroma import Chroma

vectordb = Chroma.from_documents(
    documents = split_docs,
    embedding= embedding,
    persist_directory=persist_directory  # 允许我们将persist_directory目录保存到磁盘上
)

In [ ]:
#运行 vectordb.persist 来持久化向量数据库, 以便未来使用
vectordb.persist()
#Chroma 从版本 0.4.x 开始，文档会 自动持久化，因此不再需要手动调用 vectordb.persist() 方法。
#不需要此cell中的代码

In [30]:
#print向量库中存储的数量：
vectordb._collection.count()

721

In [28]:
help(Chroma.from_documents)


Help on method from_documents in module langchain_community.vectorstores.chroma:

from_documents(documents: 'List[Document]', embedding: 'Optional[Embeddings]' = None, ids: 'Optional[List[str]]' = None, collection_name: 'str' = 'langchain', persist_directory: 'Optional[str]' = None, client_settings: 'Optional[chromadb.config.Settings]' = None, client: 'Optional[chromadb.Client]' = None, collection_metadata: 'Optional[Dict]' = None, **kwargs: 'Any') -> 'Chroma' method of abc.ABCMeta instance
    Create a Chroma vectorstore from a list of documents.
    
    If a persist_directory is specified, the collection will be persisted there.
    Otherwise, the data will be ephemeral in-memory.
    
    Args:
        collection_name (str): Name of the collection to create.
        persist_directory (Optional[str]): Directory to persist the collection.
        ids (Optional[List[str]]): List of document IDs. Defaults to None.
        documents (List[Document]): List of documents to add to the vect

In [29]:
print(dir(vectordb))


['_Chroma__query_collection', '_LANGCHAIN_DEFAULT_COLLECTION_NAME', '__abstractmethods__', '__annotations__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_asimilarity_search_with_relevance_scores', '_client', '_client_settings', '_collection', '_cosine_relevance_score_fn', '_embedding_function', '_euclidean_relevance_score_fn', '_get_retriever_tags', '_max_inner_product_relevance_score_fn', '_persist_directory', '_select_relevance_score_fn', '_similarity_search_with_relevance_scores', 'aadd_documents', 'aadd_texts', 'add_documents', 'add_images', 'add_texts', 'adelete', 'afrom_documents', 'afrom_texts', 'aget_by_ids', 'amax_marginal_relevance_search', 'a

In [36]:
question = "什么是大语言模型"

In [ ]:
#Chroma的相似度搜索使用的是余弦距离
sim_docs = vectordb.similarity_search(question, k=3)
print(f"检索到的内容数：{len(sim_docs)}")

检索到的内容数：3


In [39]:
for i, sim_doc in enumerate(sim_docs):
    print(f"检索到的第{i}个内容：\n{sim_doc.page_content[:200]}", end = "\n........................\n")

检索到的第0个内容：
现在，基于大型语言模型构建应用程序是一个非常令人兴奋和不断发展的领域。现在您已经完成了这门课程，我们认为您现在拥有了丰富的知识，可以帮助您构建其他人今天不知道如何构建的东西。因此，我希望您也能帮助我们传播并鼓励其他人也参加这门课程。

最后，希望您在完成这门课程时感到愉快，感谢您完成了这门课程。我们期待得知您构建的惊人之作。
........................
检索到的第1个内容：
随着 LLM 的发展，其大致可以分为两种类型，后续称为基础 LLM 和指令微调（Instruction Tuned）LLM。基础LLM是基于文本训练数据，训练出预测下一个单词能力的模型。其通常通过在互联网和其他来源的大量数据上训练，来确定紧接着出现的最可能的词。例如，如果你以“从前,有一只独角兽”作为 Prompt ，基础 LLM 可能会继续预测“她与独角兽朋友共同生活在一片神奇森林中”。但是，如
........................
检索到的第2个内容：
→_→
欢迎去各大电商平台选购纸质版南瓜书《机器学习公式详解第2 版》
←_←
代入刚开始的表达式, 可得本式:
q∗
j (zj) = exp (Ei̸=j[ln p(x, z)]) · exp( const )
=
exp (Ei̸=j[ln p(x, z)])
R
exp (Ei̸=j[ln p(x, z)]) dzj
实际上, 本式的分母为归一化因子, 以保证q∗
j (zj) 为概率分布
........................


In [40]:
'''MMR检索
如果只考虑检索出内容的相关性会导致内容过于单一，可能丢失重要信息。

最大边际相关性 (MMR, Maximum marginal relevance) 可以帮助我们在保持相关性的同时，增加内容的丰富度。

核心思想是在已经选择了一个相关性高的文档之后，再选择一个与已选文档相关性较低但是信息丰富的文档。
|这样可以在保持相关性的同时，增加内容的多样性，避免过于单一的结果。'''

'MMR检索\n如果只考虑检索出内容的相关性会导致内容过于单一，可能丢失重要信息。\n\n最大边际相关性 (MMR, Maximum marginal relevance) 可以帮助我们在保持相关性的同时，增加内容的丰富度。\n\n核心思想是在已经选择了一个相关性高的文档之后，再选择一个与已选文档相关性较低但是信息丰富的文档。\n|这样可以在保持相关性的同时，增加内容的多样性，避免过于单一的结果。'

In [42]:
mmr_docs = vectordb.max_marginal_relevance_search(question, k=3)

for i, sim_doc in enumerate(mmr_docs):
    print(f"MMR 检索到的第{i}个内容: \n{sim_doc.page_content[:200]}", end="\n--------------\n")

MMR 检索到的第0个内容: 
现在，基于大型语言模型构建应用程序是一个非常令人兴奋和不断发展的领域。现在您已经完成了这门课程，我们认为您现在拥有了丰富的知识，可以帮助您构建其他人今天不知道如何构建的东西。因此，我希望您也能帮助我们传播并鼓励其他人也参加这门课程。

最后，希望您在完成这门课程时感到愉快，感谢您完成了这门课程。我们期待得知您构建的惊人之作。
--------------
MMR 检索到的第1个内容: 
随着 LLM 的发展，其大致可以分为两种类型，后续称为基础 LLM 和指令微调（Instruction Tuned）LLM。基础LLM是基于文本训练数据，训练出预测下一个单词能力的模型。其通常通过在互联网和其他来源的大量数据上训练，来确定紧接着出现的最可能的词。例如，如果你以“从前,有一只独角兽”作为 Prompt ，基础 LLM 可能会继续预测“她与独角兽朋友共同生活在一片神奇森林中”。但是，如
--------------
MMR 检索到的第2个内容: 
→_→
欢迎去各大电商平台选购纸质版南瓜书《机器学习公式详解第2 版》
←_←
代入刚开始的表达式, 可得本式:
q∗
j (zj) = exp (Ei̸=j[ln p(x, z)]) · exp( const )
=
exp (Ei̸=j[ln p(x, z)])
R
exp (Ei̸=j[ln p(x, z)]) dzj
实际上, 本式的分母为归一化因子, 以保证q∗
j (zj) 为概率分布
--------------
